In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


[]

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD
from keras.models import load_model
from keras.utils import to_categorical
import math
import pickle
from sklearn.preprocessing import StandardScaler


In [5]:
index = "dowjones"
# index = "frankfurt"
with open(f"../data/{index}_calculated/absolute_periods750_250_240.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [6]:
timestep = 240
feature = 31

In [7]:
def long_short_postion(returns, k):
    position_ = np.copy(returns)
    short = np.argpartition(position_, k)[:k]
    neutral = np.argpartition(position_, len(position_) - k)[:(len(position_) - k)]
    position_[:] = 1
    position_[neutral] = 0
    position_[short] = -1
    return position_

In [8]:
positions = []
for i in range(len(dataset[0])):
    model_period = f"../model/LSTM/{index}2_absolute__1feature_period{i}.h5"
    regressor = load_model(model_period, custom_objects={"softmax": softmax})
    x_train = dataset[0][i][0].values
    scaler = StandardScaler().fit(x_train)
    
    x_test = scaler.transform(dataset[1][i][0])
    y_test = to_categorical(dataset[1][i][1].values, 2)
#     print(f"Period {i}")
#     print(f"x test shape: {x_test.shape}")
#     print(f"y test shape: {y_test.shape}")
#     print(f"x_final shape: {x_final.shape}")
#     print(f"y_final shape: {y_final.shape}")
    x_series = [x_test[i:i + timestep, j]
                for i in range(x_test.shape[0] - timestep) for j in range(feature)]
    y_series = [y_test[i + timestep, j]
                for i in range(y_test.shape[0] - timestep) for j in range(feature)]
    x_final = np.array(x_series)
    y_final = np.array(y_series)
    x_final = np.reshape(x_final, (x_final.shape[0], x_final.shape[1], 1))

    predicted =  regressor.predict(x_final)
#     predicted = np.reshape
#     print(predicted.shape)
    predicted = np.reshape(predicted[:, 1], (250, 31))
    label = predicted > 0.5
    label = label * 1 # Convert boolean to int
    print(sum(y_test[-250:, :, 1] == label).sum()/label.size)
    positions.append(predicted)

0.5104516129032258
0.48812903225806453
0.5107096774193548
0.5192258064516129
0.5370322580645162
0.5296774193548387
0.5021935483870967
0.5174193548387097
0.4967741935483871


In [9]:
stocks = pd.read_csv("../data/dowjones/all_stocks_2006-01-01_to_2018-01-01.csv", index_col='Date',parse_dates=['Date'])
stocks = stocks[["Close", "Name"]]
stocks = stocks.pivot_table(values='Close', index=stocks.index, columns='Name', aggfunc='first')
stocks.head()

Name,AABA,AAPL,AMZN,AXP,BA,CAT,CSCO,CVX,DIS,GE,...,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-03,40.91,10.68,47.58,52.58,70.44,57.80,17.45,59.08,24.40,35.37,...,26.84,10.74,23.78,58.78,45.99,61.73,56.53,30.38,46.23,58.47
2006-01-04,40.97,10.71,47.25,51.95,71.17,59.27,17.85,58.91,23.99,35.32,...,26.97,10.69,24.55,58.89,46.50,61.88,56.19,31.27,46.32,58.57
2006-01-05,41.53,10.63,47.65,52.50,70.33,59.27,18.35,58.19,24.41,35.23,...,26.99,10.76,24.58,58.70,46.95,61.69,55.98,31.63,45.69,58.28
2006-01-06,43.21,10.90,47.87,52.68,69.35,60.45,18.77,59.25,24.74,35.47,...,26.91,10.72,24.85,58.64,47.21,62.90,56.16,31.35,45.88,59.43
2006-01-09,43.42,10.86,47.08,53.99,68.77,61.55,19.06,58.95,25.00,35.38,...,26.86,10.88,24.85,59.08,47.23,61.40,56.80,31.48,45.71,59.40


In [10]:
real_returns = (stocks - stocks.shift(1)) / stocks.shift(1)
real_returns = real_returns.dropna()
real_returns[750:3000].head()

Name,AABA,AAPL,AMZN,AXP,BA,CAT,CSCO,CVX,DIS,GE,...,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-26,0.001623,0.009053,0.006610,-0.003339,0.010219,0.019327,-0.004893,0.010485,0.008640,-0.008690,...,-0.002087,0.013029,0.005291,0.002318,0.019443,0.013097,-0.000391,0.010350,-0.001623,0.018606
2008-12-29,-0.037277,0.008972,-0.045964,-0.011725,-0.013323,-0.008895,-0.015980,0.017058,-0.032011,-0.019411,...,-0.008887,-0.016077,0.011111,-0.005452,0.026136,-0.027757,0.003912,-0.001506,-0.004336,0.010753
2008-12-30,0.007576,-0.003234,0.027530,0.016949,0.031508,0.031176,0.013741,0.025577,0.047042,0.010217,...,0.020042,0.014706,0.026605,0.015282,0.030978,0.057489,0.033314,0.002716,-0.001089,0.007306
2008-12-31,0.019215,-0.011354,0.010244,0.030556,0.034424,0.023133,0.004313,0.008040,0.009342,0.024020,...,0.005171,0.026570,-0.002254,0.011453,0.006009,-0.016272,0.010558,0.020163,0.018347,0.015778
2009-01-02,0.053279,0.063167,0.060062,0.042049,0.060464,0.050146,0.040491,0.034473,0.054209,0.053704,...,0.045782,0.040000,0.031621,0.015852,0.000000,0.037218,0.025187,0.021829,0.019979,0.022673


In [11]:
all_positions = np.concatenate(positions,axis=0)
all_positions.shape

(2250, 31)

In [12]:
for k in range(1, 16):
    probabilities = pd.DataFrame(data=all_positions, index=real_returns[750:3000].index, columns=real_returns.columns)
    position = probabilities.apply(lambda x: long_short_postion(x, k), axis=1, result_type='broadcast')
    final_returns = real_returns[750:3000].mul(position)
#     no_rebalance = (final_returns + 1).product().sum()/(2 * k)
    rebalance = (1 + final_returns.sum(axis = 1)/k).product()
    rebalance1 = (1 + final_returns.sum(axis = 1)/(2 * k)).product()
#     print(f"no rebalance = {no_rebalance}")
    print(f"rebalance = {rebalance}")
    print(f"rebalance1 = {rebalance1}")

rebalance = 0.416847796589599
rebalance1 = 0.7287858977877059
rebalance = 0.988530751320795
rebalance1 = 1.058599060576066
rebalance = 2.2003725835255565
rebalance1 = 1.5484746988471223
rebalance = 1.0373938949183026
rebalance1 = 1.0532690812815328
rebalance = 1.2411430431165373
rebalance1 = 1.144256999889448
rebalance = 1.1235350642658526
rebalance1 = 1.0831576083988692
rebalance = 0.9623747025919739
rebalance1 = 0.9998904990603397
rebalance = 0.9177365598653195
rebalance1 = 0.9738202513502956
rebalance = 0.9114912909403768
rebalance1 = 0.9685717723658719
rebalance = 0.9048209816604091
rebalance1 = 0.963766259664476
rebalance = 0.8799231762933988
rebalance1 = 0.9489119605846448
rebalance = 0.8327474649310889
rebalance1 = 0.9221604112598187
rebalance = 0.8459709771506109
rebalance1 = 0.9285075023608181
rebalance = 0.8591237981217743
rebalance1 = 0.9348027933880991
rebalance = 0.8302393252717782
rebalance1 = 0.9183123644921956
